<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Packages" data-toc-modified-id="Import-Packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Packages</a></span></li><li><span><a href="#Attempt-1-to-check-for-new-contact" data-toc-modified-id="Attempt-1-to-check-for-new-contact-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Attempt 1 to check for new contact</a></span></li><li><span><a href="#Double-check-contact_check" data-toc-modified-id="Double-check-contact_check-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Double-check contact_check</a></span></li></ul></div>

### Import Packages

In [17]:
import pandas as pd
import numpy as np
import time
import os
import sqlite3
import json
pd.set_option("display.max_rows", None)
from tqdm import tqdm
from tqdm.notebook import tqdm as notebook_tqdm
import time

os.environ['WDM_LOG'] = '0'
os.environ['WDM_LOG_LEVEL'] = '0'

from fuzzysearch import find_near_matches
from thefuzz import process, fuzz

import spacy
from langdetect import detect
from deep_translator import GoogleTranslator

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common import exceptions
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup

#------------ stopword collection----------#
en = spacy.load('en_core_web_sm')
stopwords_en = en.Defaults.stop_words
de = spacy.load('de_core_news_lg')
stopwords_de = de.Defaults.stop_words
stopwords = stopwords_en.union(stopwords_de)

#------------ load custom NER model----------#
ner = spacy.load('en_core_web_lg')

d:\python_projects\thesis\lib\site-packages\thefuzz\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
workdir = 'D:\HSLU_Projects\Thesis'
if os.getcwd() != workdir:
    os.chdir(workdir)
    print(f"Working Directory is now {os.getcwd()}")
else:
    print(f"Working Directory is already set to {os.getcwd()}")

Working Directory is now D:\HSLU_Projects\Thesis


### Attempt 1 to check for new contact

In [18]:
cnx = sqlite3.connect(r"D:\HSLU_projects\Thesis\Data\Contacts.db")
df = pd.read_sql_query("SELECT * FROM Contacts", cnx)
cnx.close()

In [19]:
df.head()

,Name,Vorname,Firma,Email,Is_Valid
0,Ademoglu,Amara,Mobimo Management AG,amara.ademoglu@mobimo.ch,1
1,Aebischer,Thomas,ACUTRONIC Medical Systems AG,t.aebischer@acutronic-medical.ch,1
2,Aeschbach,André,Dialog Verwaltungs-Data AG,andre.aeschbach@dialog.ch,1
3,Agotai,Doris,FHNW,doris.agotai@fhnw.ch,1
4,Albert,Stefan,AMAG Automobil- und Motoren AG,stefan.albert@amag.ch,1


In [5]:
service = Service(executable_path=ChromeDriverManager().install())
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--log-level = 3")

In [13]:
random_slice = df.sample(n=1)
random_slice

,Name,Vorname,Firma,Email,Is_Valid
201,Helling,Felix,Crowdli,helling@crowdli.ch,1


In [14]:
def find_replacement_contact(random_slice):
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.implicitly_wait(25)
    driver.get("https://www.google.com/")
    try:
        WebDriverWait(driver, 5).until(expected_conditions.element_to_be_clickable((By.XPATH,
                                                                                    "/html/body/div[2]/div[2]/div[3]/span/div/div/div/div[3]/button[2]"))).click()
    except:
        pass
    m = driver.find_element(by=By.NAME, value="q")
    m = driver.find_element(by=By.NAME, value='q')
    m.send_keys(random_slice.iloc[0]['Firma'] + " Head of Communication Switzerland")
    m.send_keys(Keys.RETURN)

    first_page = page = driver.find_element(By .TAG_NAME, "body").text
    if "featured snippet" in first_page:
        print("Featured snippet found, extract team from here")
        try:
            WebDriverWait(driver, 5).until(expected_conditions.element_to_be_clickable((By.XPATH,
                                                                                       '//*[@id="rso"]/div[1]/block-component/div/div[1]/div/div/div/div/div[1]/div/div/div/div/div/div[2]/div/div/div[1]'))).click()
            page = driver.find_element(By .TAG_NAME, "body").text
        except:
            print("Can't find the element")
            pass
    else:
        print("not found, going to first link")

        try:
            check_for_linkedin = driver.find_element(By.XPATH, '//*[@id="rso"]/div[1]').text
            print(check_for_linkedin)
            if "https://ch.linkedin.com" in check_for_linkedin:
                print("LinkedIn found, extract info from here")
                page = check_for_linkedin
            else:
                print("LinkedIn not found, going to first link")
                WebDriverWait(driver, 5).until(expected_conditions.element_to_be_clickable((By.XPATH,
                                                                                        '//*[@id="rso"]/div[1]/div/div[1]/div/a/h3'))).click()
                time.sleep(5)
                if driver.findElement( By.XPATH, '/html/body/div[4]/div/div/div[2]/div[1]/button' ).size() != 0:
                    WebDriverWait(driver, 5).until(expected_conditions.element_to_be_clickable((By.XPATH,
                                                                                        '/html/body/div[4]/div/div/div[2]/div[1]/button'))).click()
                    page = driver.find_element(By .TAG_NAME, "body").text
                else:
                    page = driver.find_element(By .TAG_NAME, "body").text
        except:
            print("Can't find the element")
            pass

    driver.close()
    return page

In [15]:
def clean_page(page):
    soup = BeautifulSoup(page, 'html.parser')
    cleanup = soup.get_text().strip()
    jumble = cleanup.split()
    cleaned = ' '.join(jumble).strip()
    finish = []
    for word in cleaned.split():
        if word not in stopwords:
            finish.append(word)
    final = ' '.join(finish)
    tags = ner(final)
    print(soup.prettify())
    print('-'*50)
    print(final)
    print('-'*50)
    spacy.displacy.render(tags, style="ent", jupyter=True)
    print('-'*50)
    persons = []
    for ent in tags.ents:
        if ent.label_ == 'PERSON':
            print(ent.text)
            persons.append(ent.text)
    return persons[0]

In [16]:
check = find_replacement_contact(random_slice)
person = clean_page(check)
print('-'*50)
print(person)

not found, going to first link
Jens Wiesenhütter – Head of Communications Switzerland
https://ch.linkedin.com › jens-wiesen...
· Translate this page
Sankt Gallen, Schweiz · Head of Communications Switzerland · Helvetia Versicherungen Schweiz
Creative communication manager with experience and passion in strategy, social media, databased storytelling, digital marketing, content marketing, ...
Missing: Crowdli | Must include: Crowdli

Vanessa Von der Muhll – Head of Communications ... - Linkedin
https://ch.linkedin.com › vanessavondermuhll
Genf, Genf, Schweiz · Head of Communications and Engagement · ISO - International Organization for Standardization
Fifteen years' experience (largely internationally) in the field of communications including strategy, marketing, brand, web, social media, press, ...
LinkedIn found, extract info from here
Jens Wiesenhütter – Head of Communications Switzerland
https://ch.linkedin.com › jens-wiesen...
· Translate this page
Sankt Gallen, Schweiz · Head of Co

--------------------------------------------------
Jens Wiesenhütter
Sankt Gallen
Crowdli Vanessa Von Muhll – Head Communications
›
Genf
Genf
--------------------------------------------------
Jens Wiesenhütter


This process seems to be fairly consistent, but the NER model is not consistent. I will stick with this for now.

### Generating rules for creating emails

In [9]:
# find Name, Vorname, and email from head of communications
email_domain = random_slice[['Email']].iloc[0][0].split('@')[1]
print(email_domain)

kpmg.com


In [10]:
#find email address convention
current_email = random_slice[['Email']].iloc[0][0]
addressee = random_slice[['Vorname', 'Name']].iloc[0].values.tolist()
addressee = ' '.join(addressee)
print(addressee)

Lukas Marty


In [43]:
#import common umlauts
import unidecode
f = open('Data/umlautDictionary.json')
umlautDictionary = json.load(f)
f.close()


umap = {ord(key): val for key, val in umlautDictionary.items()}
print(umap)

#email address creator
def check_format(random_slice):
    first_name = random_slice[['Vorname']].iloc[0][0].lower().translate(umap)
    first_name = unidecode.unidecode(first_name)
    last_name = random_slice[['Name']].iloc[0][0].lower().translate(umap)
    last_name = unidecode.unidecode(last_name)
    last_name = last_name.replace("'", "")
    email = random_slice[['Email']].iloc[0][0]
    domain = random_slice[['Email']].iloc[0][0].split('@')[1]
    print(first_name, last_name, email)
    if len(last_name.split(' ')) > 1:
        print(last_name.split(' ')[0], last_name.split(' ')[1])
        if last_name.split(' ')[0] in email or last_name.split(' ')[1] in email:
            print("Email format is correct")
            return True
        else:
            print("Email format is incorrect")
            return False
    if first_name not in email or last_name not in email:
        print("This is a specific email format, double-checking...")
        if 'mail' in email or 'info' in email:
            print("this is the genereric info email, use this")
            return email
    format1 = first_name[0] + last_name + '@' + domain
    expression1 = "first_name[0] + last_name + '@' + domain"
    format2 = first_name + last_name[0] + '@' + domain
    expression2 = "first_name + last_name[0] + '@' + domain"
    format_3 = first_name + last_name + '@' + domain
    expression_3 = "first_name + last_name + '@' + domain"
    format_4 = first_name + '.' + last_name + '@' + domain
    expression_4 = "first_name + '.' + last_name + '@' + domain"
    format_5 = first_name + '_' + last_name + '@' + domain
    expression_5 = "first_name + '_' + last_name + '@' + domain"
    format_6 = first_name + '-' + last_name + '@' + domain
    expression_6 = "first_name + '-' + last_name + '@' + domain"
    format_7 = last_name + '.' + first_name + '@' + domain
    expression_7 = "last_name + '.' + first_name + '@' + domain"
    format_8 = last_name + '_' + first_name + '@' + domain
    expression_8 = "last_name + '_' + first_name + '@' + domain"
    format_9 = last_name + '-' + first_name + '@' + domain
    expression_9 = "last_name + '-' + first_name + '@' + domain"
    format_10 = last_name.split(' ')[0] + first_name[0] + '@' + domain
    expression_10 = "last_name.split(' ')[0] + first_name[0] + '@' + domain"
    format_11 = last_name.split(' ')[0] + first_name + '@' + domain
    expression_11 = "last_name.split(' ')[0] + first_name + '@' + domain"
    format_12 = last_name.split(' ')[0][0] + first_name + '@' + domain
    expression_12 = "last_name.split(' ')[0][0] + first_name + '@' + domain"
    format_13 = first_name + '.' + last_name.split(' ')[0] + '@' + domain
    expression_13 = "first_name + '.' + last_name.split(' ')[0] + '@' + domain"
    format_14 = first_name + '_' + last_name.split(' ')[0] + '@' + domain
    expression_14 = "first_name + '_' + last_name.split(' ')[0] + '@' + domain"
    format_15 = first_name + '-' + last_name.split(' ')[0] + '@' + domain
    expression_15 = "first_name + '-' + last_name.split(' ')[0] + '@' + domain"
    format_16 = last_name + first_name + '@' + domain
    expression_16 = "last_name + first_name + '@' + domain"
    format_17 = first_name[0] + '.' + last_name + '@' + domain
    expression_17 = "first_name[0] + '.' + last_name + '@' + domain"
    format_18 = first_name + last_name.split('-')[0] + '@' + domain
    expression_18 = "first_name + last_name.split('-')[0] + '@' + domain"
    format_19 = first_name + last_name.split('_')[0] + '@' + domain
    expression_19 = "first_name + last_name.split('_')[0] + '@' + domain"
    format_20 = first_name + "." + last_name.split('-')[0] + '@' + domain
    expression_20 = "first_name + '.' + last_name.split('-')[0] + '@' + domain"
    format_21 = first_name + "_" + last_name.split('-')[0] + '@' + domain
    expression_21 = "first_name + '_' + last_name.split('-')[0] + '@' + domain"
    format_22 = last_name + first_name[0] + '@' + domain
    expression_22 = "last_name + first_name[0] + '@' + domain"
    format_23 = last_name + "." + first_name[0] + '@' + domain
    expression_23 = "last_name + '.' + first_name[0] + '@' + domain"
    format_24 = last_name + "_" + first_name[0] + '@' + domain
    expression_24 = "last_name + '_' + first_name[0] + '@' + domain"
    format_25 = last_name + first_name + '@' + domain
    expression_25 = "last_name + first_name + '@' + domain"
    format_26 = first_name[0] + '.' + last_name[0] + '@' + domain
    expression_26 = "first_name[0] + '.' + last_name[0] + '@' + domain"
    format_27 = first_name[0] + '_' + last_name[0] + '@' + domain
    expression_27 = "first_name[0] + '_' + last_name[0] + '@' + domain"
    format_28 = last_name[0] + first_name[0] + '@' + domain
    expression_28 = "last_name[0] + first_name[0] + '@' + domain"
    format_29 = last_name[0] + '.' + first_name[0] + '@' + domain
    expression_29 = "last_name[0] + '.' + first_name[0] + '@' + domain"
    format_30 = last_name[0] + '_' + first_name[0] + '@' + domain
    expression_30 = "last_name[0] + '_' + first_name[0] + '@' + domain"
    format_31 = first_name[0] +  last_name[0] + '@' + domain
    expression_31 = "first_name[0] +  last_name[0] + '@' + domain"


    format_list = [format1, format2, format_3, format_4, format_5, format_6, format_7, format_8, format_9, format_10, format_11, format_12, format_13, format_14, format_15, format_16, format_17, format_18, format_19, format_20, format_21, format_22, format_23, format_24, format_25, format_26, format_27, format_28, format_29, format_30, format_31]
    expression_list = [expression1, expression2, expression_3, expression_4, expression_5, expression_6, expression_7, expression_8, expression_9, expression_10, expression_11, expression_12, expression_13, expression_14, expression_15, expression_16, expression_17, expression_18, expression_19, expression_20, expression_21, expression_22, expression_23, expression_24, expression_25, expression_26, expression_27, expression_28, expression_29, expression_30, expression_31]

    print("-"*100)
    for format in format_list:
        print("-"*100)
        print(f"Checking {format}")
        if format == email:
            print(f"The email format is {expression_list[format_list.index(format)]}")
            return expression_list[format_list.index(format)]

{196: 'Ae', 214: 'Oe', 220: 'Ue', 228: 'ae', 246: 'oe', 252: 'ue'}


In [95]:
random_slice = df.sample(n=1)
print(random_slice)
print("--------------------------------")
checker = check_format(random_slice)
print(checker)

         Name Vorname             Firma                 Email  Is_Valid
349  Pandolfo    Vito  CSS Versicherung  vito.pandolfo@css.ch         1
--------------------------------
vito pandolfo vito.pandolfo@css.ch
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Checking vpandolfo@css.ch
----------------------------------------------------------------------------------------------------
Checking vitop@css.ch
----------------------------------------------------------------------------------------------------
Checking vitopandolfo@css.ch
----------------------------------------------------------------------------------------------------
Checking vito.pandolfo@css.ch
The email format is first_name + '.' + last_name + '@' + domain
first_name + '.' + last_name + '@' + domain


This method is very unwieldly and I would like to find a better way to do this. However, it fits the purposes of this project. I will use it for now and if time allows I will try a more programmatic approach.

### Creating rules for umlauts

German umlauts are not allowed in the email format. I will create a list of umlauts and their replacements. Normal utf-8 normalisers do not follow the rules for german and are not suitable for this purpose.

In [14]:
import json

umlautDictionary = {u'Ä': 'Ae',
                    u'Ö': 'Oe',
                    u'Ü': 'Ue',
                    u'ä': 'ae',
                    u'ö': 'oe',
                    u'ü': 'ue'
                    }
with open('Data/umlautDictionary.json', 'w') as fp:
    json.dump(umlautDictionary, fp)

### Putting it all together
Now to construct the full workflow into a class with methods.

In [21]:
import pandas as pd
import numpy as np
import time
import os
import sqlite3
import json
pd.set_option("display.max_rows", None)
from tqdm import tqdm
from tqdm.notebook import tqdm as notebook_tqdm
import time

os.environ['WDM_LOG'] = '0'
os.environ['WDM_LOG_LEVEL'] = '0'

import spacy
import unidecode

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common import exceptions
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup

#------------ stopword collection----------#
en = spacy.load('en_core_web_sm')
stopwords_en = en.Defaults.stop_words
de = spacy.load('de_core_news_lg')
stopwords_de = de.Defaults.stop_words
stopwords = stopwords_en.union(stopwords_de)

#------------ load NER model----------#
ner = spacy.load('en_core_web_lg')

#-----------------------------------------------------------------------------------------------------------------------------------#

class ReplaceContacts():
    def __init__(self):
        self.service = Service(executable_path=ChromeDriverManager().install())
        self.chrome_options = Options()
        self.chrome_options.add_argument("--headless")
        self.chrome_options.add_argument("--log-level = 3")
        workdir = 'D:\HSLU_Projects\Thesis'
        if os.getcwd() != workdir:
            os.chdir(workdir)
        else:
            pass
        f = open(r'Data/common_acronyms.json')
        self.acronyms = json.load(f)
        f.close()
        f = open('Data/umlautDictionary.json')
        umlautDictionary = json.load(f)
        f.close()
        self.umap = {ord(key): val for key, val in umlautDictionary.items()}
        self.ner_model = spacy.load('en_core_web_lg')


    #placeholder function to load contacts, will be replaced by a function that loads the contacts the returned data of the other class
    def get_contacts(self):
        cnx = sqlite3.connect(r"D:\HSLU_projects\Thesis\Data\Contacts.db")
        df = pd.read_sql_query('SELECT * FROM Contacts', cnx)
        cnx.close()
        self.df = df

    # function to clean up text
    def clean_text(self, page):
        soup = BeautifulSoup(page, 'html.parser')
        cleanup = soup.get_text().strip()
        jumble = cleanup.split()
        cleaned = ' '.join(jumble).strip()
        finish = []
        for word in cleaned.split():
         if word not in stopwords:
            finish.append(word)
        final = ' '.join(finish)
        return final

    #function to find the contact name
    def contact_person(self, page):
        tagged_page = self.ner_model(page)
        persons = []
        for ent in tagged_page.ents:
            if ent.label_ == 'PERSON':
                persons.append(ent.text)
        person = persons[0]
        if ' - ' in person:
            person = person.split(' - ')[0]
        return person

    #function to find the correct email address format
    def create_email_rules(self, random_slice):
        umap = self.umap
        first_name = random_slice[['Vorname']].iloc[0][0].lower().translate(umap)
        first_name = unidecode.unidecode(first_name)
        last_name = random_slice[['Name']].iloc[0][0].lower().translate(umap)
        last_name = unidecode.unidecode(last_name)
        last_name = last_name.replace("'", "")
        email = random_slice[['Email']].iloc[0][0]
        domain = random_slice[['Email']].iloc[0][0].split('@')[1]
        if len(last_name.split(' ')) > 1:
            print(last_name.split(' ')[0], last_name.split(' ')[1])
            if last_name.split(' ')[0] in email or last_name.split(' ')[1] in email:
                print("Email format is correct")
                return True
            else:
                print("Email format is incorrect")
                return False
        if first_name not in email or last_name not in email:
            print("This is a specific email format, double-checking...")
            if 'mail' in email or 'info' in email:
                print("this is the genereric info email, use this")
                return email
        format1 = first_name[0] + last_name + '@' + domain
        expression1 = "first_name[0] + last_name + '@' + domain"
        format2 = first_name + last_name[0] + '@' + domain
        expression2 = "first_name + last_name[0] + '@' + domain"
        format_3 = first_name + last_name + '@' + domain
        expression_3 = "first_name + last_name + '@' + domain"
        format_4 = first_name + '.' + last_name + '@' + domain
        expression_4 = "first_name + '.' + last_name + '@' + domain"
        format_5 = first_name + '_' + last_name + '@' + domain
        expression_5 = "first_name + '_' + last_name + '@' + domain"
        format_6 = first_name + '-' + last_name + '@' + domain
        expression_6 = "first_name + '-' + last_name + '@' + domain"
        format_7 = last_name + '.' + first_name + '@' + domain
        expression_7 = "last_name + '.' + first_name + '@' + domain"
        format_8 = last_name + '_' + first_name + '@' + domain
        expression_8 = "last_name + '_' + first_name + '@' + domain"
        format_9 = last_name + '-' + first_name + '@' + domain
        expression_9 = "last_name + '-' + first_name + '@' + domain"
        format_10 = last_name.split(' ')[0] + first_name[0] + '@' + domain
        expression_10 = "last_name.split(' ')[0] + first_name[0] + '@' + domain"
        format_11 = last_name.split(' ')[0] + first_name + '@' + domain
        expression_11 = "last_name.split(' ')[0] + first_name + '@' + domain"
        format_12 = last_name.split(' ')[0][0] + first_name + '@' + domain
        expression_12 = "last_name.split(' ')[0][0] + first_name + '@' + domain"
        format_13 = first_name + '.' + last_name.split(' ')[0] + '@' + domain
        expression_13 = "first_name + '.' + last_name.split(' ')[0] + '@' + domain"
        format_14 = first_name + '_' + last_name.split(' ')[0] + '@' + domain
        expression_14 = "first_name + '_' + last_name.split(' ')[0] + '@' + domain"
        format_15 = first_name + '-' + last_name.split(' ')[0] + '@' + domain
        expression_15 = "first_name + '-' + last_name.split(' ')[0] + '@' + domain"
        format_16 = last_name + first_name + '@' + domain
        expression_16 = "last_name + first_name + '@' + domain"
        format_17 = first_name[0] + '.' + last_name + '@' + domain
        expression_17 = "first_name[0] + '.' + last_name + '@' + domain"
        format_18 = first_name + last_name.split('-')[0] + '@' + domain
        expression_18 = "first_name + last_name.split('-')[0] + '@' + domain"
        format_19 = first_name + last_name.split('_')[0] + '@' + domain
        expression_19 = "first_name + last_name.split('_')[0] + '@' + domain"
        format_20 = first_name + "." + last_name.split('-')[0] + '@' + domain
        expression_20 = "first_name + '.' + last_name.split('-')[0] + '@' + domain"
        format_21 = first_name + "_" + last_name.split('-')[0] + '@' + domain
        expression_21 = "first_name + '_' + last_name.split('-')[0] + '@' + domain"
        format_22 = last_name + first_name[0] + '@' + domain
        expression_22 = "last_name + first_name[0] + '@' + domain"
        format_23 = last_name + "." + first_name[0] + '@' + domain
        expression_23 = "last_name + '.' + first_name[0] + '@' + domain"
        format_24 = last_name + "_" + first_name[0] + '@' + domain
        expression_24 = "last_name + '_' + first_name[0] + '@' + domain"
        format_25 = last_name + first_name + '@' + domain
        expression_25 = "last_name + first_name + '@' + domain"
        format_26 = first_name[0] + '.' + last_name[0] + '@' + domain
        expression_26 = "first_name[0] + '.' + last_name[0] + '@' + domain"
        format_27 = first_name[0] + '_' + last_name[0] + '@' + domain
        expression_27 = "first_name[0] + '_' + last_name[0] + '@' + domain"
        format_28 = last_name[0] + first_name[0] + '@' + domain
        expression_28 = "last_name[0] + first_name[0] + '@' + domain"
        format_29 = last_name[0] + '.' + first_name[0] + '@' + domain
        expression_29 = "last_name[0] + '.' + first_name[0] + '@' + domain"
        format_30 = last_name[0] + '_' + first_name[0] + '@' + domain
        expression_30 = "last_name[0] + '_' + first_name[0] + '@' + domain"
        format_31 = first_name[0] +  last_name[0] + '@' + domain
        expression_31 = "first_name[0] +  last_name[0] + '@' + domain"


        format_list = [format1, format2, format_3, format_4, format_5, format_6, format_7, format_8, format_9, format_10, format_11, format_12, format_13, format_14, format_15, format_16, format_17, format_18, format_19, format_20, format_21, format_22, format_23, format_24, format_25, format_26, format_27, format_28, format_29, format_30, format_31]
        expression_list = [expression1, expression2, expression_3, expression_4, expression_5, expression_6, expression_7, expression_8, expression_9, expression_10, expression_11, expression_12, expression_13, expression_14, expression_15, expression_16, expression_17, expression_18, expression_19, expression_20, expression_21, expression_22, expression_23, expression_24, expression_25, expression_26, expression_27, expression_28, expression_29, expression_30, expression_31]
        for format in format_list:
            if format == email:
                return expression_list[format_list.index(format)]

    # function to find new contact - currently defaulting to head of communications
    def find_new_contact(self, random_slice):
        driver = webdriver.Chrome(service=self.service, options=self.chrome_options)
        driver.implicitly_wait(25)
        driver.get("https://www.google.com/")
        try:
            WebDriverWait(driver, 5).until(expected_conditions.element_to_be_clickable((By.XPATH,
                                                                                        "/html/body/div[2]/div[2]/div[3]/span/div/div/div/div[3]/button[2]"))).click()
        except:
            pass
        m = driver.find_element(by=By.NAME, value='q')
        m.send_keys(random_slice.iloc[0]['Firma'] + " Head of Communication Switzerland")
        m.send_keys(Keys.RETURN)

        first_page = page = driver.find_element(By .TAG_NAME, "body").text
        if "featured snippet" in first_page:
            print("Featured snippet found, extract team from here")
            try:
                WebDriverWait(driver, 5).until(expected_conditions.element_to_be_clickable((By.XPATH,
                                                                                           '//*[@id="rso"]/div[1]/block-component/div/div[1]/div/div/div/div/div[1]/div/div/div/div/div/div[2]/div/div/div[1]'))).click()
                page = driver.find_element(By .TAG_NAME, "body").text
            except:
                print("Can't find the element")
                pass
        else:
            print("not found, going to first link")

            try:
                check_for_linkedin = driver.find_element(By.XPATH, '//*[@id="rso"]/div[1]').text
                print(check_for_linkedin)
                if "https://ch.linkedin.com" in check_for_linkedin:
                    print("LinkedIn found, extract info from here")
                    page = check_for_linkedin
                else:
                    print("LinkedIn not found, going to first link")
                    WebDriverWait(driver, 5).until(expected_conditions.element_to_be_clickable((By.XPATH,
                                                                                            '//*[@id="rso"]/div[1]/div/div[1]/div/a/h3'))).click()
                    time.sleep(5)
                    if driver.findElement( By.XPATH, '/html/body/div[4]/div/div/div[2]/div[1]/button' ).size() != 0:
                        WebDriverWait(driver, 5).until(expected_conditions.element_to_be_clickable((By.XPATH,
                                                                                            '/html/body/div[4]/div/div/div[2]/div[1]/button'))).click()
                        page = driver.find_element(By .TAG_NAME, "body").text
                    else:
                        page = driver.find_element(By .TAG_NAME, "body").text
            except:
                print("Can't find the element")
                pass

        driver.close()
        return page

    #function to build email from found contact
    def create_email(self, random_slice, person, email_format):
        first_name = person.split(" ")[0]
        if len(person.split(" ")) == 2:
            last_name = person.split(" ")[1]
        else:
            last_name = (' '.join(person.split(" ")) - first_name)
        domain = random_slice[['Email']].iloc[0][0].split('@')[1]
        email = eval(email_format)
        return email

    #function to find and replace invalid contact
    def find_and_replace(self, df):
        random_slice = df.sample(n=1)
        print("checking for new contact for {}".format(random_slice.iloc[0]['Firma']))
        page = self.find_new_contact(random_slice)
        cleaned_page = self.clean_text(page)
        person = self.contact_person(cleaned_page)
        email_format = self.create_email_rules(random_slice)
        Email = self.create_email(random_slice, person, email_format)
        Vorname = person.split(" ")[0]
        if len(person.split(" ")) == 2:
            Name = person.split(" ")[1]
        else:
            Name = (' '.join(person.split(" ")) - Vorname)
        Firma = random_slice[['Firma']].iloc[0][0]
        print("the new contact for {} is {} {} with email {}".format(Firma, Vorname, Name, Email))
        return Name, Vorname, Email, Firma

    #function to iterate over all wrong contacts and replace with new contact
    def full_contact_replacement(self, df):
        wrong_contacts = df[df['Is_Valid'] == 0]
        for idx, row in wrong_contacts.iterrows():
            Name, Vorname, Email, Firma = self.find_and_replace(df = wrong_contacts)
            df.loc[idx, 'Name'] = Name
            df.loc[idx, 'Vorname'] = Vorname
            df.loc[idx, 'Email'] = Email
            df.loc[idx, 'Firma'] = Firma
            df.loc[idx, 'Is_Valid'] = 1
        print("{} wrong contacts replaced".format(len(wrong_contacts)))
        confirmation = input("Do you want to save the new dataframe? (y/n)")
        if confirmation == 'y':
            df = df.combine_first(wrong_contacts)
        else:
            print("Contact replacement aborted")
            pass




In [22]:
test = ReplaceContacts()
test.get_contacts()
test.find_and_replace(test.df)


checking for new contact for Pilatus-Bahnen AG
not found, going to first link
Marco Thali – Leiter Marketing - PILATUS-BAHNEN AG
https://ch.linkedin.com › ...
· Translate this page
Luzern, Luzern, Schweiz · Leiter Marketing - Communications · PILATUS-BAHNEN AG
PILATUS-BAHNEN AG · athleticum Sportmarkets AG Grafik. Head Marketing Communication · LK International AG Grafik. Marketing Coordinator · Catrade Sportmarketing AG ...
Missing: Switzerland | Must include: Switzerland
LinkedIn found, extract info from here
the new contact for Pilatus-Bahnen AG is Marco Thali with email Marco.Thali@pilatus.ch


('Marco.Thali@pilatus.ch', 'Marco', 'Thali', 'Pilatus-Bahnen AG')

## Double-check contact_check

In [15]:
import pandas as pd
import numpy as np
import time
import os
import json

pd.set_option("display.max_rows", None)
from tqdm import tqdm
import sqlite3

os.environ['WDM_LOG'] = '0'
os.environ['WDM_LOG_LEVEL'] = '0'

from fuzzysearch import find_near_matches
from thefuzz import process, fuzz

import spacy
from langdetect import detect
from deep_translator import GoogleTranslator

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common import exceptions
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

# ------------ stopword collection----------#
en = spacy.load('en_core_web_sm')
stopwords_en = en.Defaults.stop_words
de = spacy.load('de_core_news_lg')
stopwords_de = de.Defaults.stop_words
stopwords = stopwords_en.union(stopwords_de)


# ---------------------------------------------------------------------------------------------------------#

class Check_Contacts:
    def __init__(self):
        self.service = Service(executable_path=ChromeDriverManager().install())
        self.chrome_options = Options()
        self.chrome_options.add_argument("--headless")
        self.chrome_options.add_argument("--log-level = 3")
        workdir = 'D:\HSLU_Projects\Thesis'
        if os.getcwd() != workdir:
            os.chdir(workdir)
        else:
            pass
        f = open(r'Data/common_acronyms.json')
        self.acronyms = json.load(f)

    # function to connect to sqlite database
    def get_contacts(self):
        cnx = sqlite3.connect(r"D:\HSLU_projects\Thesis\Data\Contacts.db")
        df = pd.read_sql_query("SELECT * FROM Contacts", cnx)
        cnx.close()
        return df

    # function to tag loaded web elements
    def create_tags(self, text):
        ner_model = spacy.load(r"./Model/output/model-last")
        nlp = ner_model(text)
        return nlp.ents

    # function to clean html into text
    def clean_text(self, web_element):
        cleanup = web_element.text.strip()
        jumble = cleanup.split()
        cleaned = ' '.join(jumble).strip()
        return cleaned

    # function to return list of workplaces
    def list_of_firma(self, tuple):
        workplaces = []
        for i in tuple:
            if i.label_ == 'Firma':
                workplaces.append(i.text)
        return workplaces

    # function to remove stopwords
    def cleanup(self, text):
        finish = []
        try:
            for token in text.split():
                if token.lower() not in stopwords:  # checking whether the word is not
                    finish.append(token)
        except AttributeError:
            pass
        final = ' '.join(finish)
        return final

    # function to find a common abbreviation
    def acronym_checker(self, text):
        if text in self.acronyms.keys():
            return self.acronyms[text]
        else:
            return "No"

    # function to load contact table
    def load_contacts(self):
        cnx = sqlite3.connect(r"D:\HSLU_projects\Thesis\Data\Contacts.db")
        df = pd.read_sql_query("SELECT * FROM Contacts", cnx)
        cnx.close()
        self.df = df

    # function to check the link of a email google search
    def check_employment(self, df_slice):
        Top = None
        Bottom = None
        Missing = None

        driver = webdriver.Chrome(service=self.service, options=self.chrome_options)
        driver.implicitly_wait(50)
        driver.get("https://www.google.com/")
        try:
            WebDriverWait(driver, 5).until(expected_conditions.element_to_be_clickable((By.XPATH,
                                                                                        "/html/body/div[2]/div[2]/div[3]/span/div/div/div/div[3]/button[2]"))).click()
        except:
            pass
        m = driver.find_element(by=By.NAME, value="q")
        m = driver.find_element(by=By.NAME, value='q')
        if "info" not in df_slice['Email']:
            m.send_keys(df_slice['Email'])
        else:
            m.send_keys(df_slice['Vorname'] + ' ' + df_slice['Name'] + ' ' + df_slice['Firma'])
        m.send_keys(Keys.ENTER)
        time.sleep(3)
        web_element = driver.find_element(by=By.XPATH, value='//*[@id="rso"]/div[1]/div/div[1]')
        Top = self.clean_text(web_element)
        if Top is not None:
            Top = self.cleanup(Top)
        try:
            Bottom = self.clean_text(web_element=driver.find_element(by=By.XPATH, value='//*[@id="rso"]/div[1]/div/div[2]').text)
            Bottom = self.cleanup(Bottom)
        except:
            pass
        try:
            Missing = self.clean_text(web_element=driver.find_element(by=By.XPATH, value='//*[@id="rso"]/div[1]/div/div[3]').text)
            Missing = self.cleanup(Missing)
        except:
            pass
        driver.close()
        if Missing is not None and 'missing' in Missing:
            return {"Top Line": Top,
                    "Middle Line": Bottom,
                    'Missing element': Missing}
        else:
            return {"Top Line": Top,
                    "Middle Line": Bottom,
                    "Addendum": Missing}

    # function to check if email is valid
    def validation(self, n=5):
        sample_check = self.df.sample(n=n)
        with tqdm(total=n) as pbar:
            for idx, row in sample_check.iterrows():
                print(f"Checking now for {row[1]} {row[0]} at {row[2]} with {row[3]}")
                print(row)
                print("-----------------------------------------------")
                validation_online = self.check_employment(row)
                if 'Missing' in validation_online.values() or "It looks like there aren't many great matches for your search" in validation_online.values():
                    print(
                        f"{row['Vorname']} {row['Name']}'s' email {row['Email']} does not appear in the first search. They may no longer work at {row['Firma']}")
                    continue
                check_work = find_near_matches(row[2].split()[0],
                                               validation_online['Top Line'],
                                               max_l_dist=1)
                print(check_work)
                acronym = self.acronym_checker(row["Firma"])
                check_work_acronym = None
                if acronym != "No":
                    check_work_acronym = find_near_matches(acronym,
                                                           validation_online['Top Line'],
                                                           max_l_dist=1)
                    print(check_work_acronym)
                Firma_list = []
                for i in validation_online.values():
                    if i is not None:
                        tags = self.create_tags(i)
                        Firma_list.append(self.list_of_firma(tags))
                l = [item for sublist in Firma_list for item in sublist]
                check_Firma = find_near_matches(row["Firma"].split()[0], l, max_l_dist=1)
                print(l)
                if len(check_work) != 0 or check_work_acronym != None or len(check_Firma) != 0:
                    print(f"{row[1]} {row[0]} seems to still works at {row[2]}")
                elif len(check_work) != 0 and check_work_acronym != None and len(check_Firma) != 0:
                    print(f"{row[1]} {row[0]} definitely still works at {row[2]}")
                elif len(check_work) == 0 and check_work_acronym is None and len(check_Firma) == 0:
                    print(f"{row[1]} {row[0]} does not work at {row[2]} anymore")
                    row['Is_valid'] = 0
                print("-----------------------------------------------")
                pbar.update(1)


In [16]:
test = Check_Contacts()

In [17]:
test.load_contacts()

In [18]:
test.validation()

  0%|          | 0/5 [00:00<?, ?it/s]

Checking now for Gaudenz Zemp-Lüthy at Gewerbeverband Kanton Luzern with zemp@gewerbeverband-lu.ch
Name                          Zemp-Lüthy
Vorname                          Gaudenz
Firma       Gewerbeverband Kanton Luzern
Email          zemp@gewerbeverband-lu.ch
Is_Valid                               1
Name: 515, dtype: object
-----------------------------------------------
[Match(start=20, end=34, dist=0, matched='Gewerbeverband'), Match(start=152, end=166, dist=0, matched='Gewerbeverband')]


 20%|██        | 1/5 [01:21<05:26, 81.67s/it]

['Gewerbeverband Kanton Luzern', 'Luzern Direktor Gewerbeverbandes Kantons Luzern']
Gaudenz Zemp-Lüthy seems to still works at Gewerbeverband Kanton Luzern
-----------------------------------------------
Checking now for Urs Bucher at Adidas Sport GmbH with urs.bucher@adidas.ch
Name                      Bucher
Vorname                      Urs
Firma          Adidas Sport GmbH
Email       urs.bucher@adidas.ch
Is_Valid                       1
Name: 79, dtype: object
-----------------------------------------------
[Match(start=0, end=6, dist=1, matched='adidas')]


 40%|████      | 2/5 [01:29<01:55, 38.45s/it]

...

[]
Ralf Bellm seems to still works at Bettr AG
-----------------------------------------------
Checking now for Robert Lüthi at AbbVie AG with robert.luethi@abbvie.com
Name                           Lüthi
Vorname                       Robert
Firma                      AbbVie AG
Email       robert.luethi@abbvie.com
Is_Valid                           1
Name: 288, dtype: object
-----------------------------------------------
[]


 80%|████████  | 4/5 [02:34<00:31, 31.49s/it]

['CFO Advisory Digital Finance ... - Linkedin']
Robert Lüthi does not work at AbbVie AG anymore
-----------------------------------------------
Checking now for Thomas Grimm at Losinger Marazzi AG with th.grimm@losinger-marazzi.ch
Name                               Grimm
Vorname                           Thomas
Firma                Losinger Marazzi AG
Email       th.grimm@losinger-marazzi.ch
Is_Valid                               1
Name: 171, dtype: object
-----------------------------------------------
[Match(start=21, end=29, dist=0, matched='Losinger'), Match(start=50, end=58, dist=1, matched='losinger'), Match(start=116, end=124, dist=1, matched='losinger'), Match(start=257, end=265, dist=0, matched='Losinger'), Match(start=286, end=294, dist=1, matched='losinger'), Match(start=450, end=458, dist=1, matched='losinger')]


100%|██████████| 5/5 [04:23<00:00, 52.61s/it]

['Losinger Marazzi', 'Losinger Marazzi']
Thomas Grimm seems to still works at Losinger Marazzi AG
-----------------------------------------------


## Miscellaneous tests:

In [21]:
import pandas as pd

slice = df.sample(1)

In [26]:
slice.iloc[0, 2]

'first frame networkers ag'